In [65]:
""" import os
import requests
from dotenv import load_dotenv

# Load .env file
load_dotenv()
API_KEY = os.getenv("PRINTIFY_API_KEY")

BASE_URL = "https://api.printify.com/v1/"
headers = {"Authorization": f"Bearer {API_KEY}"}

response = requests.get(BASE_URL + "shops.json", headers=headers)

if response.status_code != 200:
    print(f"❌ Error: {response.status_code}")
    print(response.text)
else:
    print("✅ Success! Your shop details:")
    print(response.json())
 """

' import os\nimport requests\nfrom dotenv import load_dotenv\n\n# Load .env file\nload_dotenv()\nAPI_KEY = os.getenv("PRINTIFY_API_KEY")\n\nBASE_URL = "https://api.printify.com/v1/"\nheaders = {"Authorization": f"Bearer {API_KEY}"}\n\nresponse = requests.get(BASE_URL + "shops.json", headers=headers)\n\nif response.status_code != 200:\n    print(f"❌ Error: {response.status_code}")\n    print(response.text)\nelse:\n    print("✅ Success! Your shop details:")\n    print(response.json())\n '

In [64]:
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("PRINTIFY_API_KEY")
SHOP_ID = "23711392"  # your actual Shop ID
BASE_URL = "https://api.printify.com/v1/"
headers = {"Authorization": f"Bearer {API_KEY}"}


In [39]:

def get_all_products(shop_id):
    url = f"{BASE_URL}shops/{shop_id}/products.json"
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        raise Exception(f"Error {resp.status_code}: {resp.text}")
    return resp.json().get("data", [])

In [58]:
def summarize_variants(variants):
    """Summarize color and size info from variant titles (e.g., 'Navy / 2XL')."""
    colors, sizes = set(), set()

    for v in variants:
        # Skip inactive variants
        if not (v.get("is_enabled", True)):
            continue


        title = v.get("title", "")
        if not title:
            continue

        # Handle titles like "Navy / 2XL" or "Black - M"
        parts = [t.strip() for t in title.replace("-", "/").split("/") if t.strip()]

        if len(parts) == 1:
            # Only one part — treat as color (for single-size items)
            colors.add(parts[0])
        elif len(parts) >= 2:
            colors.add(parts[0])
            sizes.add(parts[1])

    # Return a user-friendly summary
    return f"{len(colors)} colors • {len(sizes)} sizes • Total {len([v for v in variants if v.get('is_enabled', True)])} variants"


In [53]:
def transform_products(products):
    backend_data, frontend_data = [], []

    for p in products:
        # Extract first image and first variant price
        first_image = p.get("images", [{}])[0].get("src", "")
        variants = p.get("variants", [])
        base_price = variants[0].get("price", 0) / 100 if variants else 0
        
        
          # --- BACKEND FULL STRUCTURE ---
        backend_data.append({
            "product_id": p.get("id"),
            "title": p.get("title"),
            "description": p.get("description"),
            "blueprint_id": p.get("blueprint_id"),
            "print_provider_id": p.get("print_provider_id"),
            "visible": p.get("visible"),
            "variants": variants,
            "images": p.get("images", []),
            "tags": p.get("tags", []),
            "base_price": base_price
        })

        # --- FRONTEND SUBSET ---
        frontend_data.append({
            "product_id": p.get("id"),
            "title": p.get("title"),
            "image_url": first_image,
            "price": f"${base_price:.2f}",
            "variant_summary": summarize_variants(variants),
            "status": "Published" if p.get("visible") else "Unpublished",
            "inventory": "All in stock" if p.get("visible") else "Unavailable"
        })

    return backend_data, frontend_data

In [54]:
# === Structure the data for display ===
def structure_data(products):
    """Transform raw API products into clean structured data."""
    structured = []
    for p in products:
        variants = p.get("variants", [])
        price = variants[0].get("price", 0) / 100 if variants else 0
        first_image = p.get("images", [{}])[0].get("src", "")

        structured.append({
            "Product Name": p.get("title"),
            "Product ID": p.get("id"),
            "Variant Summary": summarize_variants(variants),
            "Base Price ($)": f"${price:.2f}",
            "Status": "Unpublished",  # since you're using custom integration
            "Inventory": "All in stock",
            "Image URL": first_image,
            "Blueprint ID": p.get("blueprint_id"),
            "Print Provider ID": p.get("print_provider_id")
        })
    return structured

# === Run everything ===
products = get_all_products(SHOP_ID)
structured = structure_data(products)



In [63]:
products = get_all_products(SHOP_ID)
structured = structure_data(products)

pd.DataFrame(structured)


,Product Name,Product ID,Variant Summary,Base Price ($),Status,Inventory,Image URL,Blueprint ID,Print Provider ID
0,Copy of Unisex Oversized Boxy Tee,690f752a57ca4ce0c7094849,2 colors • 7 sizes • Total 14 variants,$30.83,Unpublished,All in stock,https://images-api.printify.com/mockup/690f752...,1382,99
1,Mouse Pad (Rectangle),6891abe9057e8b8ef20d32ef,1 colors • 1 sizes • Total 1 variants,$7.86,Unpublished,All in stock,https://images-api.printify.com/mockup/6891abe...,608,28
2,Poker Cards,68918fcf2f238abeb9027d35,1 colors • 1 sizes • Total 1 variants,$17.46,Unpublished,All in stock,https://images-api.printify.com/mockup/68918fc...,1138,94
3,Unisex Oversized Boxy Tee,68918c001505cd4c2f081bdf,2 colors • 7 sizes • Total 14 variants,$30.83,Unpublished,All in stock,https://images-api.printify.com/mockup/68918c0...,1382,99


In [50]:

# Run locally
products = get_all_products(SHOP_ID)
backend_data, frontend_data = transform_products(products)

# Save JSONs
with open("backend_full_data.json", "w") as f:
    json.dump(backend_data, f, indent=4)

with open("frontend_ready_data.json", "w") as f:
    json.dump(frontend_data, f, indent=4)

print("✅ Saved backend_full_data.json (complete API data)")
print("✅ Saved frontend_ready_data.json (UI-clean data)")

✅ Saved backend_full_data.json (complete API data)
✅ Saved frontend_ready_data.json (UI-clean data)
